# Spam dataset classification

In [2]:
import scipy.io as sio
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import math
import csv
from pylab import pcolor, show, colorbar, xticks, yticks

## Data partitioning

In [3]:
# Read in the file
train_mat = sio.loadmat("spam_data.mat")

In [4]:
# Understand the data content
train_mat.keys()

dict_keys(['training_labels', 'test_data', '__globals__', '__header__', '__version__', 'training_data'])

In [5]:
# Get the training data
train_data = train_mat['training_data']
# Get the training label
train_labels = train_mat['training_labels'][0]
# Check the amount of data in the training set
print(train_data.shape)
print(train_labels.shape)

(23702, 68)
(23702,)


In [46]:
# Shuffle the data
train_data, train_labels = shuffle(train_data, train_labels, random_state=0)

In [6]:
# Set aside 20% training sample as a validation set
size = int(math.ceil(train_data.shape[0] * 0.2))
validation_index = np.array(random.sample(range(train_data.shape[0]), size))
validation_data = train_data[validation_index]
validation_labels = train_labels[validation_index]

In [7]:
# Verify the validation set size
print(validation_data.shape )
print(validation_labels.shape)

(4741, 68)
(4741,)


In [8]:
# Construct the training set
training_data = np.delete(train_data, validation_index, 0)
training_labels = np.delete(train_labels, validation_index, 0)
print(training_data.shape)
print(training_labels.shape)

(18961, 68)
(18961,)


In [29]:
# Get the test data
test_data = train_mat['test_data']

In [10]:
# Normalize the given data
def normalize_data(data):
    normalized_data = data/np.linalg.norm(data).reshape((-1, 1))
    return normalized_data

In [30]:
# Normalize the data
norm_validation_data = normalize_data(validation_data)

norm_training_data = normalize_data(training_data)

norm_test_data = normalize_data(test_data)

In [14]:
feature_size = training_data.shape[1]
classes = 2

In [16]:
def cross_validation(training_data, training_labels, DA_fn, k_fold, constant):
    """
    Do the k-fold cross_validation with given discriminate analysis function
    and the constant that modify the covariance matrix
    It returns the average validation accuracy with the given constant
    """
    # Partition the data and labels into k-fold
    data_set = np.array(np.split(training_data, k_fold))
    labels_set = np.array(np.split(training_labels, k_fold))
    total_accuracy = []
    
    for k in range(k_fold):
        train_data = np.delete(data_set, k, 0)
        train_labels = np.delete(labels_set, k, 0)
        validation_data = data_set[k]
        validation_labels = labels_set[k]
        validation_prediction = DA_fn(train_data, train_labels, validation_data, constant)
        total_accuracy.append(np.sum(validation_prediction == validation_labels)/float(len(validation_labels)))
    accuracy = sum(total_accuracy)/len(total_accuracy)
    return accuracy

In [20]:
def find_best_const(training_data, training_labels, DA_fn, k_fold, constant_list):
    """
    Find the best constant that results in a greater accuracy in the constant list
    """
    max_accuracy = 0
    best_const = 0
    
    for const in constant_list:
        print("Testing const", const)
        current_accuracy = cross_validation(training_data, training_labels, DA_fn, k_fold, const)
        if current_accuracy > max_accuracy:
            max_accuracy = current_accuracy
            best_const = const
    return best_const

## LDA Classification

In [15]:
# Calculate the prior probability
sample_size = float(training_labels.shape[0])
prior_prob = [np.sum(training_labels == c)/sample_size for c in range(classes)]
for i, p in enumerate(prior_prob):
    print("Prior of ", i, " is ", p)

Prior of  0  is  0.492009915089
Prior of  1  is  0.507990084911


In [24]:
def LDA(training_data, training_labels, prediction_data, constant):
    # Compute the mean
    means = [np.mean(training_data[training_labels == c], axis=0) for c in range(classes)]
    # Compute the covariance matrix
    covariance = [np.cov(training_data[training_labels == c].T) for c in range(classes)]
    avg_cov = np.mean(covariance, axis=0) + np.eye(feature_size) * constant# the average covariance matrix of the 10 classes
    avg_cov_inv = np.linalg.inv(avg_cov) 
    
    # use the linear discriminant function to make prediction
    predictions = [means[c].dot(avg_cov_inv).dot(prediction_data.T) - \
                   0.5 * means[c].T.dot(avg_cov_inv).dot(means[c]) + \
                   np.log(prior_prob[c]) for c in range(classes)]
    max_prediction = np.argmax(predictions, axis=0)
    return max_prediction

In [25]:
constant_list = np.array([10**i for i in range(-10, -2)])
k_fold = 15
LDA_best_const = find_best_const(norm_training_data[:18960], training_labels[:18960], LDA, k_fold, constant_list)
print("Best constant is", LDA_best_const)

Testing const 1e-10
Testing const 1e-09
Testing const 1e-08
Testing const 1e-07
Testing const 1e-06
Testing const 1e-05
Testing const 0.0001
Testing const 0.001
Best constant is 1e-10


In [27]:
training_sizes = [100, 200, 500, 1000, 2000, 5000, 10000, norm_training_data.shape[0]]
LDA_validation_error = []
for size in training_sizes:
    LDA_validation_prediction = LDA(norm_training_data[:size], training_labels[:size],\
                                    norm_validation_data, LDA_best_const)
    error_rate = np.sum(LDA_validation_prediction != validation_labels)/float(len(validation_labels))
    LDA_validation_error.append(error_rate)

/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/core/_methods.py:68: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/function_base.py:2487: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/function_base.py:2496: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/function_base.py:2496: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)


In [28]:
LDA_validation_error

[0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.15587428812486817,
 0.13182872811643112]

Use LDA to classify the test set

In [31]:
LDA_predictions = LDA(norm_training_data, training_labels, norm_test_data, LDA_best_const)

In [33]:
# write predictions to the output file
i = 0
with open('LDA_spam_submission.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Id'] + ['Category'])
    for num in LDA_predictions:
        writer.writerow([i] + [num])
        i += 1

Using LDA, the Kaggle score is 0.85280

## QDA Classification

In [53]:
def QDA(training_data, training_labels, prediction_data, constant):
    # Compute the mean
    classes = 2
    expand_size = len(prediction_data)
    means = [np.mean(training_data[training_labels == c], axis=0) for c in range(classes)]
    # Transform the means into a list of matrix that with each row is a replica of mean
    means_transformed = [np.tile(mean.reshape(feature_size, 1), expand_size).T for mean in means]
    # Compute the covariance matrix
    covariances = [np.cov(training_data[training_labels == c].T) + np.eye(feature_size) * constant\
                   for c in range(classes)]
    cov_invs = [np.linalg.inv(cov) for cov in covariances] # inverse of covariance matrix
    
    # use the discriminant function to make prediction
    predictions = [-0.5 * (prediction_data - means_transformed[c]).dot(cov_invs[c])\
                   .dot((prediction_data - means_transformed[c]).T).diagonal() - \
                   0.5 * np.tile(np.log(np.linalg.norm(cov_invs[c])), expand_size) + \
                   np.tile(np.log(prior_prob[c]), expand_size) for c in range(classes)]
    max_prediction = np.argmax(predictions, axis=0)
    return max_prediction

In [54]:
constant_list = np.array([10**i for i in range(-20, -2)])
k_fold = 15
QDA_best_const = find_best_const(norm_training_data[:18960], training_labels[:18960], QDA, k_fold, constant_list)
print("Best constant is", LDA_best_const)

Testing const 1e-20
Testing const 1e-19
Testing const 1e-18
Testing const 1e-17
Testing const 1e-16
Testing const 1e-15
Testing const 1e-14
Testing const 1e-13
Testing const 1e-12
Testing const 1e-11
Testing const 1e-10
Testing const 1e-09
Testing const 1e-08
Testing const 1e-07
Testing const 1e-06
Testing const 1e-05
Testing const 0.0001
Testing const 0.001
Best constant is 1e-10


In [55]:
QDA_validation_error = []
for size in training_sizes:
    QDA_validation_prediction = QDA(norm_training_data[:size], training_labels[:size],\
                                    norm_validation_data, QDA_best_const)
    error_rate = np.sum(QDA_validation_prediction != validation_labels)/float(len(validation_labels))
    QDA_validation_error.append(error_rate)

/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/core/_methods.py:68: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/function_base.py:2487: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/function_base.py:2496: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/numpy/lib/function_base.py:2496: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)


In [56]:
QDA_validation_error

[0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.50263657456232858,
 0.17274836532377136,
 0.15756169584475849]

Use QDA to classify the test set

In [57]:
QDA_predictions = QDA(norm_training_data, training_labels, norm_test_data, QDA_best_const)

In [58]:
# write predictions to the output file
i = 0
with open('QDA_spam_submission.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Id'] + ['Category'])
    for num in QDA_predictions:
        writer.writerow([i] + [num])
        i += 1

Using QDA, the classification score is 0.83540